# LSTM Open Food Facts

## Imports

In [1]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 6.8MB/s 
     |████████████████████████████████| 1.2MB 13.2MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 204kB 34.9MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
     |████████████████████████████████| 61kB 9.7MB/s 
Mounted at /content/gdrive


In [2]:
#hide
from fastbook import *
from IPython.display import display,HTML

In [3]:
from fastai.text.all import *

In [4]:
path = r'/content'

## Test sur 10K samples

In [7]:
df = pd.read_csv('data_for_nlp.csv', low_memory=False, sep=',')
df = df.sample(10000)

In [20]:
dls_lm = TextDataLoaders.from_df(path=path, df=df, text_col='text', label_col='label', is_lm=True)
dls_lm.show_batch(max_n=3)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,text,text_
0,xxbos xxmaj framboises ( cat . xxup i ) . xxmaj framboises bio xxbos xxup vesi . xxup xxunk . xxup xxunk xxup ja xxup suola . xxmaj xxunk xxmaj xxunk xxbos xxmaj corn syrup ; sugar ; water ; modified corn starch ; gum arabic ; artificial flavor ; carnauba wax ; shellac ; citric acid ; malic acid ; titanium dioxide ( as color ) ; red 40 ; yellow,xxmaj framboises ( cat . xxup i ) . xxmaj framboises bio xxbos xxup vesi . xxup xxunk . xxup xxunk xxup ja xxup suola . xxmaj xxunk xxmaj xxunk xxbos xxmaj corn syrup ; sugar ; water ; modified corn starch ; gum arabic ; artificial flavor ; carnauba wax ; shellac ; citric acid ; malic acid ; titanium dioxide ( as color ) ; red 40 ; yellow 5
1,; xxmaj xxunk : xxmaj citronensäure ) ; xxmaj weizenkleber ; xxmaj backtriebmittel : xxmaj diphosphate ; xxmaj natriumcarbonate ; xxmaj xxunk ; xxmaj ethyl xxmaj alkohol ; xxmaj zucker ; xxmaj speisesalz ; xxmaj mehlbehandlungsmittel : xxmaj ascorbinsäure . 6 xxmaj xxunk xxbos xxmaj bread flour ( enriched flour ; bleached [ flour ; niacin ; iron ; thiamine mononitrate ; riboflavin ; folic acid ] ) ; red bean filling,xxmaj xxunk : xxmaj citronensäure ) ; xxmaj weizenkleber ; xxmaj backtriebmittel : xxmaj diphosphate ; xxmaj natriumcarbonate ; xxmaj xxunk ; xxmaj ethyl xxmaj alkohol ; xxmaj zucker ; xxmaj speisesalz ; xxmaj mehlbehandlungsmittel : xxmaj ascorbinsäure . 6 xxmaj xxunk xxbos xxmaj bread flour ( enriched flour ; bleached [ flour ; niacin ; iron ; thiamine mononitrate ; riboflavin ; folic acid ] ) ; red bean filling ;
2,enzymes ) ; organic dehydrated onions ; organic cultured dextrose ; organic cane juice ; organic garlic ; organic spice ; organic canola oil ; organic dehydrated carrots ; organic potato flour . xxmaj organic white cheddar cauliflower soup xxbos xxmaj carbonated water ; sugar ; orange juice concentrate ; apple juice concentrate ; citric acid ; natural flavors ; rebaudioside a ( stevia leaf extract ) . xxmaj h - e,) ; organic dehydrated onions ; organic cultured dextrose ; organic cane juice ; organic garlic ; organic spice ; organic canola oil ; organic dehydrated carrots ; organic potato flour . xxmaj organic white cheddar cauliflower soup xxbos xxmaj carbonated water ; sugar ; orange juice concentrate ; apple juice concentrate ; citric acid ; natural flavors ; rebaudioside a ( stevia leaf extract ) . xxmaj h - e -


### Language Model

In [21]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [22]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.146273,2.787494,0.477325,16.240274,00:22


### Saving and Loading Models

In [23]:
learn.save('1epoch')

Path('/content/models/1epoch.pth')

In [24]:
learn = learn.load('1epoch')

In [25]:
learn.unfreeze()
learn.fit_one_cycle(5, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,2.542958,2.571611,0.501311,13.086896,00:24
1,2.325241,2.473281,0.513479,11.861296,00:24
2,2.096078,2.295792,0.541864,9.932301,00:24
3,1.834581,2.270813,0.551101,9.687268,00:24
4,1.666618,2.283904,0.552098,9.814924,00:23


In [26]:
learn.save_encoder('finetuned')

### Classification

In [28]:
dls = TextDataLoaders.from_df(path=path, df=df, text_col='text', label_col='label', is_lm=False, vocab=dls_lm.vocab)
dls.show_batch(max_n=3)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,text,category
0,xxbos u xxmaj xxunk xxunk instant xxunk ; xxunk : xxmaj xxunk t xxup xxunk ; xxunk ; 6 ; xxunk xxmaj xxunk ; cukor ; xxunk xxunk . xxmaj xxunk ( xxunk ) maltodextrin ; xxunk ( xxunk - xxunk ; xxunk - xxunk xxunk - xxunk ) ; so ; ( xxunk és xxup xxunk xxunk ( xxunk ; xxunk xxunk ( 3 % ) ; xxunk ( 2 % ) ; xxmaj cayenne - bors ( xxunk ) ; i m a ; xxunk bors xxunk xxup te xxup xxunk ; xxunk ( xxunk ) . a xxunk xxunk ; xxunk és xxunk xxunk xxup xxunk xxunk xxunk ar vistas xxunk . xxmaj xxunk : xxunk xxunk xxunk ( 25 xxup kviešu milti ; xxunk xxunk ; xxunk ; xxunk xxunk xxunk ; cukurs ; rapšu xxunk ; xxunk xxunk . xxmaj xxunk xxunk ( 15 %,cereals and potatoes
1,xxbos xxup ingredientes ( chile ) : xxmaj harina xxmaj integral de xxmaj avena ( 59 % ) ; xxmaj fibra xxmaj soluble de xxmaj maiz . xxmaj sémola de xxmaj trigo ( 14 % ) ; xxmaj semolina de xxmaj maiz ( 4 % ) ; xxmaj salvado de xxmaj trigo ( 2 % ) ; xxmaj inulina ; xxmaj maltitol ; xxmaj yogurt xxmaj xxunk ( 0;9 % ) ; xxmaj fosfato xxmaj tricálcico ; xxmaj sal ; xxmaj carbonato de xxmaj calcio ; xxmaj dióxido de xxmaj xxunk ; xxmaj saborizante xxmaj idéntico a xxmaj natural ; xxmaj maltodextrina ; xxmaj lecitina de xxmaj soya ; ácido xxmaj láctico ; xxmaj estevia ; xxmaj vitamina xxup c ; xxmaj niacina ; xxmaj antioxidante xxup bht ; xxunk de xxmaj zinc ; xxmaj pirofosfato xxmaj xxunk ; xxmaj sucralosa ; xxmaj saborizante xxmaj natural ; xxmaj vitamina xxup,cereals and potatoes
2,xxbos xxup xxunk xxmaj xxunk an xxunk xxup xxunk xxmaj xxunk l xxunk - xxunk xxunk xxup do xxmaj xxunk xxunk xxunk de xxmaj xxunk ho xxunk % ) ; / xxmaj xxunk 1 g ) ; ne d xxmaj xxunk xxunk xxmaj xxunk t xxunk xxmaj xxunk xxmaj apfel ; xxmaj orange ; xxmaj ananas ; xxunk ; xxmaj xxunk ; r xxunk xxunk - xxup e414 ; xxunk xxunk xxunk e xxmaj xxunk xxmaj xxunk für xxunk xxunk ; / xxmaj xxunk ; ( davon 7 xxunk ) - xxunk ; xxmaj xxunk g xxmaj e160a ; xxmaj xxunk : min 8 % . xxmaj das xxunk et xxmaj xxunk ; xxmaj xxunk ; xxmaj xxunk ; xxmaj xxunk xxunk xxunk mg ( 7;5 % & ' ) / xxmaj folsäure 15 ug ( 7;5 % * ) ; xxunk ( 7 xxup xxunk xxmaj xxunk - .,beverages


In [29]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

In [30]:
learn = learn.load_encoder('finetuned')

### Fine-Tuning

In [31]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.110636,0.855312,0.719000,00:12


In [32]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.923454,0.726325,0.763000,00:13


In [33]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.759044,0.679995,0.787500,00:14


In [34]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.579148,0.644136,0.806500,00:15
1,0.535337,0.637978,0.804500,00:15


In [35]:
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.516851,0.633910,0.804000,00:15
1,0.466217,0.627625,0.816000,00:14


In [36]:
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.431010,0.619938,0.816500,00:15
1,0.407581,0.611569,0.822000,00:15


## Classification sur 100K samples

In [38]:
df_full = pd.read_csv('data_for_nlp.csv', low_memory=False, sep=',')
df_full.shape

(442623, 2)

In [39]:
df_full = df_full.sample(100000)

In [41]:
dls_lm = TextDataLoaders.from_df(path=path, df=df_full, text_col='text', label_col='label', is_lm=True)
dls_lm.show_batch(max_n=3)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,text,text_
0,xxbos xxmaj epinards ; sel . épinards hachés xxbos xxmaj farine de _ blé _ - purée de tomates mi - réduite - eau - _ mozarella _ 9;6 % - jambon cuit choix fumé 8;7 % ( viande de porc - sel - dextrose - sucre - stabilisants : diphosphates ; triphosphates - antioxydants : ascorbate de sodium ; extraits de romarin - conservateur : nitrite de sodium ) - _,xxmaj epinards ; sel . épinards hachés xxbos xxmaj farine de _ blé _ - purée de tomates mi - réduite - eau - _ mozarella _ 9;6 % - jambon cuit choix fumé 8;7 % ( viande de porc - sel - dextrose - sucre - stabilisants : diphosphates ; triphosphates - antioxydants : ascorbate de sodium ; extraits de romarin - conservateur : nitrite de sodium ) - _ edam
1,culture ; oleoresin of paprika ; sodium ascorbate ; flavoring ; sodium nitrite ; bha ; bht ; citric acid ) ; green peppers ; onion ; red pepper ; black olives ( olives ; water ; salt ; ferrous gluconate ) . xxmaj rising crust ; supreme pizza xxbos _ lait _ pasteurisé de vache ; sel ; ferments lactiques et d'affinage ; chlorure de calcium ; coagulant xxup e235 .,; oleoresin of paprika ; sodium ascorbate ; flavoring ; sodium nitrite ; bha ; bht ; citric acid ) ; green peppers ; onion ; red pepper ; black olives ( olives ; water ; salt ; ferrous gluconate ) . xxmaj rising crust ; supreme pizza xxbos _ lait _ pasteurisé de vache ; sel ; ferments lactiques et d'affinage ; chlorure de calcium ; coagulant xxup e235 . xxmaj
2,xxmaj filet de poulet xxbos xxmaj provolone cheese ( pasteurized milk ; cheese culture ; salt ; enzymes ) . xxmaj deli style not smoked provolone cheese slices ; provolone xxbos xxmaj xxunk xxmaj corse ; sucre de canne roux ' xxunk % ) ; jus de citron ' ; pectine de fruit si nécessaire . xxmaj issu de l'agriculture biologique . xxmaj préparée avec 60 g de fruit pour 100 g,filet de poulet xxbos xxmaj provolone cheese ( pasteurized milk ; cheese culture ; salt ; enzymes ) . xxmaj deli style not smoked provolone cheese slices ; provolone xxbos xxmaj xxunk xxmaj corse ; sucre de canne roux ' xxunk % ) ; jus de citron ' ; pectine de fruit si nécessaire . xxmaj issu de l'agriculture biologique . xxmaj préparée avec 60 g de fruit pour 100 g .


In [42]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [43]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,2.422909,2.283332,0.543993,9.809312,04:24


In [44]:
learn.fit_one_cycle(3, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,2.273290,2.246858,0.548014,9.457971,04:23
1,2.256840,2.205354,0.553390,9.073465,04:25
2,2.195039,2.197421,0.554748,9.001764,04:24


In [45]:
learn.save_encoder('finetuned')

In [46]:
dls = TextDataLoaders.from_df(path=path, df=df_full, text_col='text', label_col='label', is_lm=False, vocab=dls_lm.vocab)
dls.show_batch(max_n=3)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,text,category
0,xxbos xxup latte scremato reidratato ; cialda 18 % ( farina di cocco ; sciroppo di glucosio ; albume d'uovo in polvere da galline xxup latte in polvere ; emulsionanti : mono - e digliceridi degli acidi grassi ; grassi ; addensante : alginato di sodio ; gelatina alimentare ( origine xxup frumento ; zucchero ; olio di cocco ; emulsionanti : lecitine ( soia ) ; allevate all'aperto 0;1 % ) ; siero di xxup latte in polvere ; emulsionanti : addensanti : alginato di sodio ; gelatina alimentare ( origine bovina ) . bovina ) ; aromi . xxmaj può contenere tracce di xxup uova e altra xxup frutta a sciroppo di zucchero xxunk ; sale ) ; copertura al cacao magro 13 % [ oli vegetali ( olio di cocco ; olio di semi di girasole ) ; zucchero ; cacao gelatina alimentare ( origine bovina ),milk and dairy products
1,xxbos xxmaj mini - feuilleté apéritif garni d'une préparation au saumon 29 % : xxmaj pâte feuilletée 65 % * - xxmaj garniture 35 % * : xxmaj eau ; farine de blé ; margarine ( huiles et graisses végétales de palme et de colza ; eau ; sel ; émulsifiant : mono - et diglycérides d'acides gras ; correcteur d'acidité : acide citrique ) ; saumon fumé 5;5 % ( saumon xxmaj atlantique xxmaj salmo salar ; sel ) ; préparation déshydratée pour sauce ( lactosérum ; amidon modifié ; lait entier en poudre ; sel ; épaississant : gomme xanthane ; poivre ; muscade ) ; beurre ; chapelure ( farine de blé ; eau ; sel ; levure ) ; sel ; persil ; oeuf entier en poudre ; aneth 0;06 % ; poivre noir ; estragon . xxmaj mini - feuilleté apéritif garni d'une préparation aux,composite foods
2,xxbos xxup gb xxmaj delicate wafer rolls half - coated in dark chocolate ( 28 % ) . xxmaj ingredients : xxmaj wheat flour ; sugar ; vegetable fat ( palm ) ; cocoa mass ; cocoa butter ; skimmed milk powder ; whey products ( milk ) ; hen 's egg yolk powder ; emulsifier : lecithins ; salt ; whole milk powder ; raising agents : sodium carbonates ; spices ; flavourings . xxmaj may contain : soya . xxmaj made in xxmaj germany . xxmaj store in a cool ; dry place away from heat . xxmaj best before : see side of pack : xxup xxunk . xxrep 4 y . xxmaj imported and distributed by : xxmaj bahlsen xxup xxunk ; xxmaj xxunk xxmaj house ; xxmaj xxunk xxmaj park ; xxmaj xxunk xxmaj hill ; xxmaj xxunk xxmaj st xxmaj peter ; xxmaj xxunk,sugary snacks


In [47]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

In [48]:
learn = learn.load_encoder('finetuned')

In [49]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.837615,0.575109,0.815600,01:59


In [50]:
learn.fit_one_cycle(3, 2e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.794661,0.553018,0.819950,02:01
1,0.771995,0.527547,0.829250,02:02
2,0.751581,0.527932,0.829600,02:00
